In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("D:\OneDrive - NITT\Custom_Download\dataset_train.csv")

In [3]:
df.isna().mean()*100

query          0.000000
intent         0.000000
sentiment     99.796204
misspelled     0.000000
entity1        2.076169
entity2        7.107375
entity3       52.388231
entity4       93.988027
entity5       99.668832
entity6       99.949051
entity7       99.987263
dtype: float64

In [4]:
df = df[['query','intent','entity1','entity2','entity3']]

In [5]:
df['intent'].value_counts()

BROWSE       5768
GOTO         1930
GIBBERISH     141
OTHER          12
Name: intent, dtype: int64

In [6]:
df['entity1'][0]

'genre_modifier(entype=genre,entities=sports,lang=eng,value=sports)'

# Anomaly detection

In [7]:
# df[:10].loc[3]['entity2']

In [8]:
# df[df['entity1'].apply(lambda x:x.startswith('cost_modifier'))]['entity1']

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

data = df

data.fillna('', inplace=True)

data['combined'] = data['query'] + ' ' + data['intent'] + ' ' + data['entity1'] + ' ' + data['entity2'] + ' '+data['entity3']

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(data['combined'])


In [10]:
from sklearn.ensemble import IsolationForest

# Train the Isolation Forest model

for i in range(1,5):
    model = IsolationForest(random_state=42,contamination=0.1*i)
    model.fit(X)
    # Predict anomalies
    data[f'anomaly{i}'] = model.predict(X)

    # -1 for anomalies, 1 for normal instances
    data[f'anomaly{i}'] = data[f'anomaly{i}'].apply(lambda x: 'Anomaly' if x == -1 else 'Normal')

In [11]:
# data['anomaly'].value_counts()

In [12]:
temp = data.copy()
for col in ['anomaly1','anomaly2','anomaly3','anomaly4']:
    temp = temp[temp[col]=='Anomaly']

In [13]:
temp

,query,intent,entity1,entity2,entity3,combined,anomaly1,anomaly2,anomaly3,anomaly4
21,English for the Braves and the Phillies,BROWSE,"locale_modifier(entype=language,entities=Engli...","sport_team(entype=sportsTeam,entities=Atlanta ...","sport_team(entype=sportsTeam,entities=Philadel...",English for the Braves and the Phillies BROWSE...,Anomaly,Anomaly,Anomaly,Anomaly
27,Show me Major League Baseball playoff games,BROWSE,"sports_league(entype=league,entities=MLB,entit...",,,Show me Major League Baseball playoff games BR...,Anomaly,Anomaly,Anomaly,Anomaly
28,Astros vs. the Twins,BROWSE,"sport_team(entype=sportsTeam,entities=Minnesot...","sport_team(entype=sportsTeam,entities=Houston ...",,Astros vs. the Twins BROWSE sport_team(entype=...,Anomaly,Anomaly,Anomaly,Anomaly
30,Philly vs. the Braves,BROWSE,"sport_team(entype=sportsTeam,entities=Atlanta ...","sport_team(entype=sportsTeam,entities=Philadel...",,Philly vs. the Braves BROWSE sport_team(entype...,Anomaly,Anomaly,Anomaly,Anomaly
31,Braves vs. the Philadelphia Phillies,BROWSE,"sport_team(entype=sportsTeam,entities=Philadel...","sport_team(entype=sportsTeam,entities=Atlanta ...",,Braves vs. the Philadelphia Phillies BROWSE sp...,Anomaly,Anomaly,Anomaly,Anomaly
...,...,...,...,...,...,...,...,...,...,...
7759,Cula de Freddy en español la película de Fredd...,GOTO,"locale_modifier(entype=language,entities=spani...","content_type(entype=type_modifier,entities=mov...","program_name(entype=movie,entities=Freddy,lang...",Cula de Freddy en español la película de Fredd...,Anomaly,Anomaly,Anomaly,Anomaly
7765,Película de terror en español,BROWSE,"locale_modifier(entype=language,entities=spani...","genre_modifier(entype=genre,entities=horror,la...","content_type(entype=type_modifier,entities=mov...",Película de terror en español BROWSE locale_mo...,Anomaly,Anomaly,Anomaly,Anomaly
7785,Free new movies,BROWSE,"content_type(entype=type_modifier,entities=mov...","temporal_modifier(entype=air_modifier,entities...","cost_modifier(entype=cost_modifier,entities=fr...",Free new movies BROWSE content_type(entype=typ...,Anomaly,Anomaly,Anomaly,Anomaly
7793,Results for New York,BROWSE,"sport_team(entype=sportsTeam,lang=eng,value=Ne...",,,Results for New York BROWSE sport_team(entype=...,Anomaly,Anomaly,Anomaly,Anomaly
